# Two Body Problem Relative to an Inertial Frame

Solves the inertial two-body problem in three dimensions using the RKF4(5) method.

### Import libraries and modules

In [3]:
import numpy as np
from scipy.integrate import solve_ivp
import numpy.linalg as linalg

### Set up initial conditions

In [4]:
mass = np.array([1e26, 1e26])

R1_0 = np.array([0, 0, 0])
R2_0 = np.array([3000, 0, 0])
V1_0 = np.array([10, 20, 30])
V2_0 = np.array([0, 40, 0])

y0 = np.concatenate((R1_0, R2_0, V1_0, V2_0))

tspan = np.array([0, 480])

### Main function

In [ ]:
def two_body(mass, y0, tspan):
    # Gravitational constant
    G = 6.67259e-20
    
    # Position vectors of m1 and m2
    R1 = y0[0:3]
    R2 = y0[3:6]
    
    # Velocity vectors of m1 and m2
    V1 = y0[6:9]
    V2 = y0[9:12]
    
    # Acceleration vectors of m1 and m2
    r = linalg.norm(R2-R1)
    A1 = G*mass[1]*(R2-R1)/(r**3)
    A2 = G*mass[0]*(R2-R1)/(r**3)
    
    # Integrate equations of motion
    dydt = [V1, V2, A1, A2]
    answer = solve_ivp(dydt, tspan, y0, method="RK45")
    
    return answer

### Run function

In [ ]:
answer = two_body(mass, y0, tspan)
answer